###  Домашнее задание по NER: извлечение адресов с помощью библиотеки [Natasha](https://github.com/natasha)

### Загрузка необходимых библиотек

In [216]:
import wget, os, re
from pprint import pprint
from natasha import AddressExtractor
from ipymarkup import show_markup
from yargy.pipelines import (
    pipeline,
    morph_pipeline)
from yargy.predicates import (
    eq, in_, dictionary,normalized,
    type, gram
)
from yargy.tokenizer import MorphTokenizer
from yargy import interpretation as interp
from yargy import rule, Parser, or_
from yargy.interpretation import fact, attribute

#### Загрузка файла, из которого будем извлекать информацию

In [330]:
wget.download('https://github.com/sjut/DPO_Materials/blob/master/Практические занятия/flats.txt')

'flats.txt'

In [4]:
flats = open('flats.txt', encoding = 'utf-8').read()

#### Запустим извлечение адресов из нашего текста про квартиры c помощью готовой библиотеки AddrX: 

In [331]:
#Инициализируем экстрактор для адресов:
AddrX =  AddressExtractor()

In [5]:
ad_matches = AddrX(flats)

Смотрю, что извлекла встроенная библиотека для извлечения адресов из текста flats:

In [7]:
for match in ad_matches:
    print(match.span, match.fact)

[70, 96) Address(parts=[Street(name='Староконюшенный', type='переулок'), Building(number='30', type='дом')])
[307, 325) Address(parts=[Street(name='Арбат', type='улица'), Building(number='15/43', type='дом')])
[532, 553) Address(parts=[Street(name='Б.Грузинская', type='улица'), Building(number='14', type='дом')])
[703, 727) Address(parts=[Street(name='Аптекарский', type='переулок'), Building(number='3/22', type='дом')])
[1343, 1366) Address(parts=[Street(name='Спартаковская', type='улица'), Building(number='18', type='дом')])
[1547, 1568) Address(parts=[Street(name='Госпитальный', type='переулок'), Building(number='8', type='дом')])
[1747, 1764) Address(parts=[Street(name='М.Почтовая', type='улица'), Building(number='10', type=None)])
[1999, 2019) Address(parts=[Street(name='Токмаков', type='переулок'), Building(number='28/16', type=None)])
[2213, 2248) Address(parts=[Street(name='Б. Почтовая', type='улица'), Building(number='18/20', type='дом'), Building(number='16', type='корпус')])


#### А теперь попробую сама написать разные правила

In [27]:
#Сделаем список текстов и сразу удалим пустые строки в тексте:
flats_list = list(filter(None, flats.splitlines()))

In [9]:
#pprint(flats_list)
for el in flats_list:
    print(el)

text1. ann1. Арбатская м. 1-комн. кв-ру, или м.Кропоткинская 7 мин/п, Староконюшенный пер., д.30. 35/21 кв.м, кухня 10, 10/12 эт, блоч. дома, балкон, тел., хор. сост., с/у совм., окна в тихий сквер, освобождение сразу, продаю, 45 тыс.$. Торг Т 557-00-73 
text2. ann2. Арбатская м. 1-комн. кв ру, 5 м/пеш., ул.Арбат дом 15/43; 53/20/13 кв.м, еврорем., 4/6-эт. кирп. дома, д/фон, евростиль, тел , кондиц., стоянка, с мебелью, охрана, свободна, хозяин, 130 тыс.$. Торг. Т.762-55-67, Сергей
text3. ann3. Баррикадная м. 1-комн. кв-ру, ул.Б.Грузинская, д.14, 36/20.2 кв.м, кухня 8.3, 2/14-эт. кирп. дома, балкон, тел., консьержка, свободна, Т.962-30-63 
text4. ann4. Бауманская м. 1-комн. кв-ру, 5 мин/п., Аптекарский пер., д.3/22, 30/16.2 кв.м, кухня 8, вые. бельэтаж/8-эт, кирп. дома, тел., с/у совм. больш., хор. сост., хор. двор, тихо, свободную, 27 тыс.$. Т.737-76-67, раб., Т.265-23-31, дом., Аня 
text5. ann5. Бауманская м. 1-комн. кв-ру, метро пешком, 45/22 кв.м, кухня 10, лоджия 3, 8/8-эт. кирп. 

Создадим всякие переменные, которые помогают компактно обращаться к нужным инструментам:

In [888]:
INT = type('INT') #предикат type будет мэтчить по типу данных - в нашем случае INT
DOT = eq('.')  #чтобы быстро обращаться к точке
COMMA = eq(',') #чтобы быстро обращаться к запятой
DASH = in_('-–—--') #всякие виды тире, дефиса
ADJF = gram('ADJF') #чтобы быстро обращаться к прилагательным

Инициализирую морф парсер: Yargy парсит с помощью Pymorphy2

In [889]:
#MorphTokenizer принимает на вход строку (текст) и возвращает список токенов.
TOKENIZER = MorphTokenizer() 

Напишем функцию для показа мэтчей и деревьев разбора (Update: на деревья пока не буду смотреть - включу только показ факта и подсветку)

In [821]:
#У парсера есть несколько методов. По аналогии с модулем re  - есть метод findall, который возвращает список мэтчей:
#Каждый match выводится как последовательность токенов. У каждого объекта мэтча есть span - span начала и конец мэтча
#посмотрим на них отдельно:
#очень часто приходится смотреть на спаны, поэтому в Наташе есть отдельная библиотека для этого, которая принимает на вход текст, в нашем случае line со списками спанов
# и показывает в Jupiter желтым квадратиком разметку

def show_matches (rule, *lines):
    parser = Parser(rule)
    for line in lines:
        matches = parser.findall(line)
        matches = sorted(matches, key=lambda _: _.span)
        spans = [_.span for _ in matches]
        show_markup(line, spans)
        if matches:
            facts = [_.fact for _ in matches]
            if len(facts) == 1:
                facts = facts[0]
                display(facts)
            #match = matches[0]
            #display(match.tree.as_dot)
            #display(match.fact)

### Извлечем названия станций метро

In [328]:
#Загрузим список станций Московского метрополитена (для учебных целей ограничимся пока только московским метрополитеном...)
wget.download('https://github.com/ChernayaAnastasia/Python-Course_HSE/blob/master/Yargy parser/metro_stations.txt')

'metro_stations.txt'

Названия станций метро - это один из элементов списка stations (подстраховка, чтобы не извлекать все подряд)

In [846]:
stations = []
with open('metro_stations.txt', encoding = 'utf-8') as file:
    for line in file:
        station = re.sub('\u2028', '', line).rstrip()
        stations.append(station)
#pprint(stations)

Создадим факт Metro_Station с полем name

In [847]:
Metro_Station = fact(
    'Metro_Station',
    ['name'] 
                    )

In [848]:
#Для обработки названий станций метро передаем в morph_pipeline список названий станций:
METRO_NAME = morph_pipeline(stations)

#Для обработки "метро":
ABBRV = morph_pipeline(['м',
                        'ст.м',
                        'ст. м',
                        'метро',
                        'станция метро'
                       ])

#Правило для обработки "метро" в разных вариациях, в том числе с точкой и без: 
METRO = rule(ABBRV,
           DOT.optional()
            )

#Правило для извлечения названий станций метро в тексте: 
METRO_STATION = or_(
    rule(
        METRO_NAME.interpretation(
                Metro_Station.name),
        METRO),

    rule(
        METRO,
        METRO_NAME.interpretation(
                Metro_Station.name)
        )
                ).interpretation(
                Metro_Station)

In [849]:
show_matches(METRO_STATION, *flats_list)

BoxMarkup('text1. ann1. Арбатская м. 1-комн. кв-ру, или м.Кропоткинская 7 мин/п, Староконюшенный пер., д.30. 35/21 кв.м, кухня 10, 10/12 эт, блоч. дома, балкон, тел., хор. сост., с/у совм., окна в тихий сквер, освобождение сразу, продаю, 45 тыс.$. Торг Т 557-00-73 ',
          [Span(13, 25, None), Span(45, 60, None)])

BoxMarkup('text2. ann2. Арбатская м. 1-комн. кв ру, 5 м/пеш., ул.Арбат дом 15/43; 53/20/13 кв.м, еврорем., 4/6-эт. кирп. дома, д/фон, евростиль, тел , кондиц., стоянка, с мебелью, охрана, свободна, хозяин, 130 тыс.$. Торг. Т.762-55-67, Сергей',
          [Span(13, 25, None)])

Metro_Station(name='Арбатская')

BoxMarkup('text3. ann3. Баррикадная м. 1-комн. кв-ру, ул.Б.Грузинская, д.14, 36/20.2 кв.м, кухня 8.3, 2/14-эт. кирп. дома, балкон, тел., консьержка, свободна, Т.962-30-63 ',
          [Span(13, 27, None)])

Metro_Station(name='Баррикадная')

...

Metro_Station(name='Белорусская')

BoxMarkup('text19. ann19. Белорусская м. 1-комн. кв-ру, ул. Бутырский вал, д. 34, 10 м/п; 36/20 кв.м, к. 10, 12/12-пан. дома, с/у совм., окна двор, тел., балкон, хор. сост., домофон, свободна, 28 тыс.$. Торг. Т.299-96-51',
          [Span(15, 29, None)])

Metro_Station(name='Белорусская')

BoxMarkup('text20. ann20. Белорусская м. 1-комн. кв-ру, 6 мин.п., Б.Тишинский пер., 22/15 кв.м, 2/6 эт. кирп, дома, паркет, пот.3.15, домофон, тихо, 18 тыс.$, торг, без посредников. Альтернатива. Т.253-30-92, вечером',
          [Span(15, 29, None)])

Metro_Station(name='Белорусская')

### НЕРЕШЕННАЯ ПРОБЛЕМА:  
В первом тексте есть два упоминания станций метро. В итоге ни один атрибут name не заполняется. Как устранить эту проблему?

### Пробую извлечь количество комнат

In [833]:
Rooms_Number = fact(
    'Rooms_Number',
    ['number'] 
                    )

Немного посмотрим на морф разборы нужных нам конструкций

In [55]:
list(TOKENIZER('1-комн. кв-ру'))

[Token('1', [0, 1), 'INT'),
 Token('-', [1, 2), 'PUNCT'),
 MorphToken('комн',
            [2, 6),
            'RU',
            [Form('комна', Grams(NOUN,Name,anim,femn,gent,plur)),
             Form('комна', Grams(NOUN,Name,accs,anim,femn,plur))]),
 Token('.', [6, 7), 'PUNCT'),
 MorphToken('кв', [8, 10), 'RU', [Form('кв', Grams(UNKN))]),
 Token('-', [10, 11), 'PUNCT'),
 MorphToken('ру', [11, 13), 'RU', [Form('ру', Grams(UNKN))])]

In [56]:
list(TOKENIZER('1-комнатную квартиру'))

[Token('1', [0, 1), 'INT'),
 Token('-', [1, 2), 'PUNCT'),
 MorphToken('комнатную',
            [2, 11),
            'RU',
            [Form('комнатный', Grams(ADJF,accs,femn,sing))]),
 MorphToken('кв', [12, 14), 'RU', [Form('кв', Grams(UNKN))]),
 Token('-', [14, 15), 'PUNCT'),
 MorphToken('ру', [15, 17), 'RU', [Form('ру', Grams(UNKN))])]

In [57]:
list(TOKENIZER('однокомнатную квартиру'))

[MorphToken('однокомнатную',
            [0, 13),
            'RU',
            [Form('однокомнатный', Grams(ADJF,accs,femn,sing))]),
 MorphToken('квартиру',
            [14, 22),
            'RU',
            [Form('квартира', Grams(NOUN,accs,femn,inan,sing))])]

In [834]:
MODIFIERS = {
    'однокомнатный': 1,
    'двухкомнатный': 2,
    'трехкомнатный': 3,
    'четырехкомнатный': 4,
    'пятикомнатный': 5
            }

MODIFIER = dictionary(MODIFIERS).interpretation(interp.normalized().custom(MODIFIERS.get))
    
ROOMS_NUMBER = or_(
    rule(INT.interpretation(Rooms_Number.number.custom(int)),
        eq('-').optional(), # 1 комн.
        eq('комн'), #1-комн.
        DOT.optional()
        ),
    rule(INT.interpretation(Rooms_Number.number.custom(int)), #1-комнатную
        DASH.optional(), #1 комнатную
        normalized('комнатный')
        ),
    rule(MODIFIER).interpretation(Rooms_Number.number)
                ).interpretation(Rooms_Number)

In [841]:
show_matches(ROOMS_NUMBER, *flats_list)

BoxMarkup('text1. ann1. Арбатская м. 1-комн. кв-ру, или м.Кропоткинская 7 мин/п, Староконюшенный пер., д.30. 35/21 кв.м, кухня 10, 10/12 эт, блоч. дома, балкон, тел., хор. сост., с/у совм., окна в тихий сквер, освобождение сразу, продаю, 45 тыс.$. Торг Т 557-00-73 ',
          [Span(26, 33, None)])

Rooms_Number(number=1)

BoxMarkup('text2. ann2. Арбатская м. 1-комн. кв ру, 5 м/пеш., ул.Арбат дом 15/43; 53/20/13 кв.м, еврорем., 4/6-эт. кирп. дома, д/фон, евростиль, тел , кондиц., стоянка, с мебелью, охрана, свободна, хозяин, 130 тыс.$. Торг. Т.762-55-67, Сергей',
          [Span(26, 33, None)])

Rooms_Number(number=1)

BoxMarkup('text3. ann3. Баррикадная м. 1-комн. кв-ру, ул.Б.Грузинская, д.14, 36/20.2 кв.м, кухня 8.3, 2/14-эт. кирп. дома, балкон, тел., консьержка, свободна, Т.962-30-63 ',
          [Span(28, 35, None)])

...

Rooms_Number(number=1)

BoxMarkup('text19. ann19. Белорусская м. 1-комн. кв-ру, ул. Бутырский вал, д. 34, 10 м/п; 36/20 кв.м, к. 10, 12/12-пан. дома, с/у совм., окна двор, тел., балкон, хор. сост., домофон, свободна, 28 тыс.$. Торг. Т.299-96-51',
          [Span(30, 37, None)])

Rooms_Number(number=1)

BoxMarkup('text20. ann20. Белорусская м. 1-комн. кв-ру, 6 мин.п., Б.Тишинский пер., 22/15 кв.м, 2/6 эт. кирп, дома, паркет, пот.3.15, домофон, тихо, 18 тыс.$, торг, без посредников. Альтернатива. Т.253-30-92, вечером',
          [Span(30, 37, None)])

Rooms_Number(number=1)

In [217]:
show_matches(ROOMS_NUMBER,
            'двухкомнатную квартиру',
            '2 комн квартиру',
            '3-комнатную квартиру')

BoxMarkup('двухкомнатную квартиру', [Span(0, 13, None)])

Rooms_Number(number=2)

BoxMarkup('2 комн квартиру', [Span(0, 6, None)])

Rooms_Number(number=2)

BoxMarkup('3-комнатную квартиру', [Span(0, 11, None)])

Rooms_Number(number=3)

### Пробуем извлечь расстояние до метро

In [835]:
Distance_from_metro = fact(
    'Distance_from_metro',
    ['Walking_Distance', 'Time', 'Time_Unit'] 
                        )

In [107]:
list(TOKENIZER('7 мин/п'))

[Token('7', [0, 1), 'INT'),
 MorphToken('мин',
            [2, 5),
            'RU',
            [Form('мина', Grams(NOUN,femn,gent,inan,plur))]),
 Token('/', [5, 6), 'PUNCT'),
 MorphToken('п',
            [6, 7),
            'RU',
            [Form('плата', Grams(Abbr,Fixd,NOUN,femn,inan,nomn,sing)),
             Form('плата', Grams(Abbr,Fixd,NOUN,femn,gent,inan,sing)),
             Form('плата', Grams(Abbr,Fixd,NOUN,datv,femn,inan,sing)),
             Form('плата', Grams(Abbr,Fixd,NOUN,accs,femn,inan,sing)),
             Form('плата', Grams(Abbr,Fixd,NOUN,ablt,femn,inan,sing)),
             Form('плата', Grams(Abbr,Fixd,NOUN,femn,inan,loct,sing)),
             Form('плата', Grams(Abbr,Fixd,NOUN,femn,inan,nomn,plur)),
             Form('плата', Grams(Abbr,Fixd,NOUN,femn,gent,inan,plur)),
             Form('плата', Grams(Abbr,Fixd,NOUN,datv,femn,inan,plur)),
             Form('плата', Grams(Abbr,Fixd,NOUN,accs,femn,inan,plur)),
             Form('плата', Grams(Abbr,Fixd,NOUN,ablt,femn,

In [108]:
list(TOKENIZER('5 м/пеш.'))

[Token('5', [0, 1), 'INT'),
 MorphToken('м',
            [2, 3),
            'RU',
            [Form('метр', Grams(Abbr,Fixd,NOUN,gent,inan,masc,sing)),
             Form('метр', Grams(Abbr,Fixd,NOUN,inan,masc,nomn,sing)),
             Form('метр', Grams(Abbr,Fixd,NOUN,ablt,inan,masc,sing)),
             Form('метр', Grams(Abbr,Fixd,NOUN,gent,inan,masc,plur)),
             Form('метр', Grams(Abbr,Fixd,NOUN,datv,inan,masc,sing)),
             Form('метр', Grams(Abbr,Fixd,NOUN,accs,inan,masc,sing)),
             Form('метр', Grams(Abbr,Fixd,NOUN,inan,loct,masc,sing)),
             Form('метр', Grams(Abbr,Fixd,NOUN,inan,masc,nomn,plur)),
             Form('метр', Grams(Abbr,Fixd,NOUN,datv,inan,masc,plur)),
             Form('метр', Grams(Abbr,Fixd,NOUN,accs,inan,masc,plur)),
             Form('метр', Grams(Abbr,Fixd,NOUN,ablt,inan,masc,plur)),
             Form('метр', Grams(Abbr,Fixd,NOUN,inan,loct,masc,plur)),
             Form('москва',
                  Grams(Abbr,Fixd,Geox,NOUN,Sgtm,

In [109]:
list(TOKENIZER('метро пешком'))

[MorphToken('метро',
            [0, 5),
            'RU',
            [Form('метро', Grams(Fixd,NOUN,inan,neut,nomn,sing)),
             Form('метро', Grams(Fixd,NOUN,gent,inan,neut,sing)),
             Form('метро', Grams(Fixd,NOUN,datv,inan,neut,sing)),
             Form('метро', Grams(Fixd,NOUN,accs,inan,neut,sing)),
             Form('метро', Grams(Fixd,NOUN,ablt,inan,neut,sing)),
             Form('метро', Grams(Fixd,NOUN,inan,loct,neut,sing)),
             Form('метро', Grams(Fixd,NOUN,inan,neut,nomn,plur)),
             Form('метро', Grams(Fixd,NOUN,gent,inan,neut,plur)),
             Form('метро', Grams(Fixd,NOUN,datv,inan,neut,plur)),
             Form('метро', Grams(Fixd,NOUN,accs,inan,neut,plur)),
             Form('метро', Grams(Fixd,NOUN,ablt,inan,neut,plur)),
             Form('метро', Grams(Fixd,NOUN,inan,loct,neut,plur))]),
 MorphToken('пешком', [6, 12), 'RU', [Form('пешком', Grams(ADVB))])]

In [110]:
list(TOKENIZER('5 м.п.'))

[Token('5', [0, 1), 'INT'),
 MorphToken('м',
            [2, 3),
            'RU',
            [Form('метр', Grams(Abbr,Fixd,NOUN,gent,inan,masc,sing)),
             Form('метр', Grams(Abbr,Fixd,NOUN,inan,masc,nomn,sing)),
             Form('метр', Grams(Abbr,Fixd,NOUN,ablt,inan,masc,sing)),
             Form('метр', Grams(Abbr,Fixd,NOUN,gent,inan,masc,plur)),
             Form('метр', Grams(Abbr,Fixd,NOUN,datv,inan,masc,sing)),
             Form('метр', Grams(Abbr,Fixd,NOUN,accs,inan,masc,sing)),
             Form('метр', Grams(Abbr,Fixd,NOUN,inan,loct,masc,sing)),
             Form('метр', Grams(Abbr,Fixd,NOUN,inan,masc,nomn,plur)),
             Form('метр', Grams(Abbr,Fixd,NOUN,datv,inan,masc,plur)),
             Form('метр', Grams(Abbr,Fixd,NOUN,accs,inan,masc,plur)),
             Form('метр', Grams(Abbr,Fixd,NOUN,ablt,inan,masc,plur)),
             Form('метр', Grams(Abbr,Fixd,NOUN,inan,loct,masc,plur)),
             Form('москва',
                  Grams(Abbr,Fixd,Geox,NOUN,Sgtm,

In [112]:
list(TOKENIZER('7 мин. пеш.'))

[Token('7', [0, 1), 'INT'),
 MorphToken('мин',
            [2, 5),
            'RU',
            [Form('мина', Grams(NOUN,femn,gent,inan,plur))]),
 Token('.', [5, 6), 'PUNCT'),
 MorphToken('пеш', [7, 10), 'RU', [Form('пеший', Grams(ADJS,masc,sing))]),
 Token('.', [10, 11), 'PUNCT')]

In [113]:
list(TOKENIZER('10 мин.п'))

[Token('10', [0, 2), 'INT'),
 MorphToken('мин',
            [3, 6),
            'RU',
            [Form('мина', Grams(NOUN,femn,gent,inan,plur))]),
 Token('.', [6, 7), 'PUNCT'),
 MorphToken('п',
            [7, 8),
            'RU',
            [Form('плата', Grams(Abbr,Fixd,NOUN,femn,inan,nomn,sing)),
             Form('плата', Grams(Abbr,Fixd,NOUN,femn,gent,inan,sing)),
             Form('плата', Grams(Abbr,Fixd,NOUN,datv,femn,inan,sing)),
             Form('плата', Grams(Abbr,Fixd,NOUN,accs,femn,inan,sing)),
             Form('плата', Grams(Abbr,Fixd,NOUN,ablt,femn,inan,sing)),
             Form('плата', Grams(Abbr,Fixd,NOUN,femn,inan,loct,sing)),
             Form('плата', Grams(Abbr,Fixd,NOUN,femn,inan,nomn,plur)),
             Form('плата', Grams(Abbr,Fixd,NOUN,femn,gent,inan,plur)),
             Form('плата', Grams(Abbr,Fixd,NOUN,datv,femn,inan,plur)),
             Form('плата', Grams(Abbr,Fixd,NOUN,accs,femn,inan,plur)),
             Form('плата', Grams(Abbr,Fixd,NOUN,ablt,femn

In [870]:
TIME_UNITS = {'мин': 'минута',
              'мин.': 'минута',
            'м': 'минута',
            'м.': 'минута',
            'минута': 'минута',
            'ч': 'час',
            'ч.': 'час'
             }

MEANS = {'пешком': 'да',
         'пеш.': 'да',
         'пеш': 'да',
         'п': 'да',
         'п.': 'да',
        'п-': 'да',  #10 мин.п-.ул.Волочаевская, д.2, корп.1
         'на общественном транспорте': 'нет',
         'на общ. транспорте': 'нет',
         'на общ транспорте': 'нет',
         'на автомобиле': 'нет'}

TIME_UNIT = morph_pipeline(TIME_UNITS).interpretation(interp.normalized().custom(TIME_UNITS.get))

IS_WALKING_DISTANCE = morph_pipeline(MEANS).interpretation(Distance_from_metro.Walking_Distance.custom(MEANS.get))
    
DISTANCE_FROM_METRO = or_(
                        rule(INT.interpretation(Distance_from_metro.Time.custom(int)),
                            TIME_UNIT.interpretation(Distance_from_metro.Time_Unit),
                            in_('/').optional(),
                               rule (IS_WALKING_DISTANCE).interpretation(Distance_from_metro.Walking_Distance)
                            ),
    
                        rule(eq('метро'), 
                             rule(IS_WALKING_DISTANCE).interpretation(Distance_from_metro.Walking_Distance)
                            ) #метро пешком
                        ).interpretation(Distance_from_metro)

In [871]:
show_matches(DISTANCE_FROM_METRO, *flats_list)

BoxMarkup('text1. ann1. Арбатская м. 1-комн. кв-ру, или м.Кропоткинская 7 мин/п, Староконюшенный пер., д.30. 35/21 кв.м, кухня 10, 10/12 эт, блоч. дома, балкон, тел., хор. сост., с/у совм., окна в тихий сквер, освобождение сразу, продаю, 45 тыс.$. Торг Т 557-00-73 ',
          [Span(61, 68, None)])

Distance_from_metro(Walking_Distance='да',
                    Time=7,
                    Time_Unit='минута')

BoxMarkup('text2. ann2. Арбатская м. 1-комн. кв ру, 5 м/пеш., ул.Арбат дом 15/43; 53/20/13 кв.м, еврорем., 4/6-эт. кирп. дома, д/фон, евростиль, тел , кондиц., стоянка, с мебелью, охрана, свободна, хозяин, 130 тыс.$. Торг. Т.762-55-67, Сергей',
          [Span(41, 49, None)])

Distance_from_metro(Walking_Distance='да',
                    Time=5,
                    Time_Unit='минута')

BoxMarkup('text3. ann3. Баррикадная м. 1-комн. кв-ру, ул.Б.Грузинская, д.14, 36/20.2 кв.м, кухня 8.3, 2/14-эт. кирп. дома, балкон, тел., консьержка, свободна, Т.962-30-63 ',
          [])

...

BoxMarkup('text18. ann18. Белорусская м. 1-комн. кв-ру, ул.Верхняя, д.6;43/20 кв.м, кухня 13,2/14-эт.кирп. дома ЖСК, тел., лоджия 6 заст., с/у совм., паркет, темная комната 2.7, домоф., мет .дверь, консьерж., окна во двор, 41.5 тыс.$, торг. Т.405-85-64 ',
          [])

BoxMarkup('text19. ann19. Белорусская м. 1-комн. кв-ру, ул. Бутырский вал, д. 34, 10 м/п; 36/20 кв.м, к. 10, 12/12-пан. дома, с/у совм., окна двор, тел., балкон, хор. сост., домофон, свободна, 28 тыс.$. Торг. Т.299-96-51',
          [Span(71, 77, None)])

Distance_from_metro(Walking_Distance='да',
                    Time=10,
                    Time_Unit='минута')

BoxMarkup('text20. ann20. Белорусская м. 1-комн. кв-ру, 6 мин.п., Б.Тишинский пер., 22/15 кв.м, 2/6 эт. кирп, дома, паркет, пот.3.15, домофон, тихо, 18 тыс.$, торг, без посредников. Альтернатива. Т.253-30-92, вечером',
          [Span(45, 53, None)])

Distance_from_metro(Walking_Distance='да',
                    Time=6,
                    Time_Unit='минута')

In [872]:
show_matches(DISTANCE_FROM_METRO, 
             '8 минут пешком',
             '8 мин. пешком',
             '2 часа пешком',
             '7 мин/п',
             '5 м/пеш.',
             'метро пешком',
             '5 м.п.',
             '7 мин. пеш.',
             '10 мин.п',
             '5 минут на общественном транспорте', 
            )

BoxMarkup('8 минут пешком', [Span(0, 14, None)])

Distance_from_metro(Walking_Distance='да',
                    Time=8,
                    Time_Unit='минута')

BoxMarkup('8 мин. пешком', [Span(0, 13, None)])

Distance_from_metro(Walking_Distance='да',
                    Time=8,
                    Time_Unit='минута')

BoxMarkup('2 часа пешком', [Span(0, 13, None)])

...

Distance_from_metro(Walking_Distance='да',
                    Time=7,
                    Time_Unit='минута')

BoxMarkup('10 мин.п', [Span(0, 8, None)])

Distance_from_metro(Walking_Distance='да',
                    Time=10,
                    Time_Unit='минута')

BoxMarkup('5 минут на общественном транспорте', [Span(0, 34, None)])

Distance_from_metro(Walking_Distance='нет',
                    Time=5,
                    Time_Unit='минута')

### Попытка самостоятельно (без готовой библиотеки) извлечь адреса

#### Загрузим список улиц российских городов. 
Список исходно взят [отсюда]('https://raw.githubusercontent.com/AMDmi3/streetmangler/master/data/ru/main.txt').

Для наших учебных этот список очень широкий. Чтобы код работал быстрее (под конкретные "московские" объявления), я его в ячейках ниже буду сокращать.
Мне кажется, часть работы со списком в данном коде можно было бы оптимизировать (слишком много ячеек с кодом про него я написала), 
но поскольку нет времени сейчас, я оставлю так - оптимизирую позже.

In [329]:
wget.download('https://github.com/ChernayaAnastasia/Python-Course_HSE/blob/master/Yargy parser/streets.txt')

'streets.txt'

In [790]:
streets = open('streets.txt', encoding = 'utf-8').readlines()

In [753]:
len(streets)

42845

In [718]:
streets

['# vim: tabstop=29 expandtab softtabstop=29 syntax=conf\n',
 '# Основной список (потенциально всех) русских названий улиц\n',
 '# ========================================================================\n',
 '# Для удобства:\n',
 '# - Не удаляйте заголовок\n',
 '# - Поддерживайте сортировку списка (см. скрипт sort.sh)\n',
 '# - Добавляйте комментарии к неочевидным названиям (Ангелов переулок,\n',
 '#   Усачёва улица) и проблемам с е/ё; в комментарии указывайте регион,\n',
 '#   где название встречается и, если возможно, ссылки на информацию\n',
 '#   почему название именно такое\n',
 '# - Комментарии начинаются с # <- 30-й позиции в строке\n',
 '# - Возможные ошибки, которые надо расследовать и исправить, отмечаются\n',
 '#   XXX или FIXME\n',
 '# - Для названий, проверенных по внешним источникам, добавляйте тэги\n',
 '#   в комментариях. При изменении, соответственно, тэг следует убрать.\n',
 '#   На данный момент:\n',
 '#     [ОМКУМ] - классификатор улиц Москвы\n',
 '#     Реестр на

Пробуем сократить наш большой список, отсеив не московские улицы по комментарию #:

In [791]:
for st in streets:
    if re.search(r'#.+', st) is not None and re.search(r'Москва', st) is None:
           streets.remove(st)

In [755]:
len(streets)

33326

In [687]:
streets

['# Основной список (потенциально всех) русских названий улиц\n',
 '# Для удобства:\n',
 '# - Поддерживайте сортировку списка (см. скрипт sort.sh)\n',
 '#   Усачёва улица) и проблемам с е/ё; в комментарии указывайте регион,\n',
 '#   почему название именно такое\n',
 '# - Возможные ошибки, которые надо расследовать и исправить, отмечаются\n',
 '# - Для названий, проверенных по внешним источникам, добавляйте тэги\n',
 '#   На данный момент:\n',
 '#     Реестр названий объектов городской среды Правительства Санкт-Петербурга\n',
 '#     [СПБ4]  - таблица 4 (названия проездов)\n',
 '# Общие непонятки:\n',
 '# - Все Ново-Какойтовские улицы вероятно должны быть Новокакойтовскими\n',
 '0-я линия\n',
 '0-я улица\n',
 '1-е Корытовское шоссе\n',
 '1-е Успенское шоссе\n',
 '1-й Авдотьинский переулок\n',
 '1-й Авиационный проезд\n',
 '1-й Автодромный переулок\n',
 '1-й Автозаводский проезд\n',
 '1-й Автомобильный проезд\n',
 '1-й Автосборочный переулок\n',
 '1-й Аграрный переулок\n',
 '1-й Агрегат

In [792]:
#Удаляем ВСЕ комментарии из списка
my_streets = [(re.sub(r'#.+', '',street)) for street in streets]
#Удаляем пустые строки и лишние пробелы и новые строки по краям: 
my_streets = [st.strip() for st in my_streets if st != '\n'] 

In [757]:
my_streets

['0-я линия',
 '0-я улица',
 '1-е Корытовское шоссе',
 '1-е Успенское шоссе',
 '1-й Авдотьинский переулок',
 '1-й Авиационный проезд',
 '1-й Автодромный переулок',
 '1-й Автозаводский проезд',
 '1-й Автомобильный проезд',
 '1-й Автосборочный переулок',
 '1-й Аграрный переулок',
 '1-й Агрегатный переулок',
 '1-й Агрономический переулок',
 '1-й Адыгейский переулок',
 '1-й Азовский проезд',
 '1-й Академический переулок',
 '1-й Академический проезд',
 '1-й Акмолинский проезд',
 '1-й Акуловский проезд',
 '1-й Александровский проезд',
 '1-й Алеутский переулок',
 '1-й Аллейный проезд',
 '1-й Алмазный переулок',
 '1-й Алтайский переулок',
 '1-й Алтуховский переулок',
 '1-й Амбулаторный проезд',
 '1-й Амурский переулок',
 '1-й Амурский проезд',
 '1-й Амурский тупик',
 '1-й Ангарский проезд',
 '1-й Андреевский переулок',
 '1-й Андреевский проезд',
 '1-й Аникинский переулок',
 '1-й Апельсиновый тупик',
 '1-й Аптечный проезд',
 '1-й Артезианский проезд',
 '1-й Артельный переулок',
 '1-й Артельный 

In [759]:
len(my_streets)

33314

In [793]:
"М." in my_streets

False

Добавим в наши список дополнительно улицы с сокращенными прилагательными типа Большая, Малая, Нижняя, чтобы мэтчить случаи типа: Б. Грузинская

In [794]:
# Большая Грузинская -> Б. Грузинская
#НО не трогаем случаи типа: Большая улица, иначе получим улицу с названием М. 

for street in my_streets:
    if re.search(r'^Больш(?:ой|ая)(?!\n| улица| переулок| проезд| шоссе| тупик| проспект| набережная| аллея)', street) is not None: #Проверяем, что у нас при этом не начало и не конец строки, то есть Большая - это не название улицы
        new_street = re.sub(r'Больш(?:ой|ая)', 'Б.', street)
        my_streets.append(new_street)
    if re.search (r'^Мал(?:ый|ая)(?!\n| улица| переулок| проезд| шоссе| тупик| проспект| набережная| аллея)', street) is not None:
        new_street = re.sub(r'Мал(?:ый|ая)', 'М.', street)
        my_streets.append(new_street)
    if re.search (r'^Нижн(?:яя|ий)(?!\n| улица| переулок| проезд| шоссе| тупик| проспект| набережная| аллея)', street) is not None:
        new_street = re.sub(r'Нижн(?:яя|ий)', 'Н.', street)
        my_streets.append(new_street)

In [795]:
#Проверка, что код выше сработал: 
for st in my_streets:
    if re.match(r'Б\.', st) is not None:
           print(st)

Б. Азовская улица
Б. Академическая улица
Б. Алтайская улица
Б. Андроньевская улица
Б. Арбековская улица
Б. Армавирская улица
Б. Базарная улица
Б. Боргустанская улица
Б. Боровая улица
Б. Бронная улица
Б. Бульварная улица
Б. Бутовская улица
Б. Витаминная улица
Б. Власьевская улица
Б. Внуковская улица
Б. Вольская улица
Б. Воробьёвская улица
Б. Восточная улица
Б. Гвардейская улица
Б. Горная улица
Б. Гражданская улица
Б. Грузинская улица
Б. Дачная улица
Б. Декабрьская улица
Б. Демидовская улица
Б. Донецкая улица
Б. Дорогомиловская улица
Б. Екатерининская улица
Б. Заводская улица
Б. Закаменская улица
Б. Заливная улица
Б. Заозёрная улица
Б. Запрудная улица
Б. Заречная улица
Б. Затонская улица
Б. Зеленовская улица
Б. Зеленогорская улица
Б. Зелёная улица
Б. Зуевская улица
Б. Ивановская улица
Б. Инженерная улица
Б. Казанская улица
Б. Казачья улица
Б. Калитниковская улица
Б. Калужская улица
Б. Каменная улица
Б. Караванная улица
Б. Клинцевская улица
Б. Кожевенная улица
Б. Колхозная улица
Б. Кольце

In [796]:
"М." in my_streets

False

Удаляем слова "улица", "переулок" и т.д. из названий в списке: грубо считаем, что в объявлениях их нет как части названия улицы: ул. 1905 года (?ул. Улица 1905 года)

In [797]:
#Проверяем, что 'улица' и подобные  или в начале или в конце, чтобы не трогать случаи типа 1-я улица 8 Марта:
my_streets = [(re.sub(r'^(?:улица|переулок|проезд|шоссе|тупик|проспект|набережная|аллея) | (?:улица|переулок|проезд|шоссе|тупик|проспект|набережная|аллея)$', '', street)).strip() for street in my_streets]

In [798]:
my_streets

['0-я линия',
 '0-я',
 '1-е Корытовское',
 '1-е Успенское',
 '1-й Авдотьинский',
 '1-й Авиационный',
 '1-й Автодромный',
 '1-й Автозаводский',
 '1-й Автомобильный',
 '1-й Автосборочный',
 '1-й Аграрный',
 '1-й Агрегатный',
 '1-й Агрономический',
 '1-й Адыгейский',
 '1-й Азовский',
 '1-й Академический',
 '1-й Академический',
 '1-й Акмолинский',
 '1-й Акуловский',
 '1-й Александровский',
 '1-й Алеутский',
 '1-й Аллейный',
 '1-й Алмазный',
 '1-й Алтайский',
 '1-й Алтуховский',
 '1-й Амбулаторный',
 '1-й Амурский',
 '1-й Амурский',
 '1-й Амурский',
 '1-й Ангарский',
 '1-й Андреевский',
 '1-й Андреевский',
 '1-й Аникинский',
 '1-й Апельсиновый',
 '1-й Аптечный',
 '1-й Артезианский',
 '1-й Артельный',
 '1-й Артельный',
 '1-й Артиллерийский',
 '1-й Архангельский',
 '1-й Архангельский',
 '1-й Архивный',
 '1-й Асеевский',
 '1-й Астровый',
 '1-й Аткарский',
 '1-й Атмосферный',
 '1-й Аэродромный',
 '1-й Аэрофлотский',
 '1-й Бабьегородский',
 '1-й Базарный',
 '1-й Базовый',
 '1-й Байкальский',
 '1

In [799]:
"М." in my_streets

False

Проверяем, есть ли конкретная улица в списке:

In [709]:
"Староконюшенный" in my_streets

True

In [800]:
"М." in my_streets

False

В ходе тестирования выяснилось, что в списке нет улиц Нижняя, Верхняя, Бутырский вал - пока просто добавлю

In [590]:
my_streets.append('Нижняя')

In [805]:
"Нижняя" in my_streets

True

In [593]:
my_streets.append('Верхняя')

In [594]:
"Верхняя" in fine_streets

True

In [600]:
my_streets.append('Волховский')

In [803]:
"Бутырский вал" in my_streets

False

In [804]:
my_streets.append('Бутырский вал')

In [808]:
"Бутырский вал" in my_streets

True

In [802]:
'1-я Малая' in my_streets

True

In [809]:
'Кропоткинская'in my_streets

True

In [810]:
"""
Костыль! Так делать нельзя! В Мск нет такой улицы, зато есть метро с омонимичным названием. 
Нужно уметь в правиле прописать, что до токена с названием улицы не может идти токен, маркирующий "метро".
"""
my_streets.remove("Кропоткинская")

### Напишем правило для извлечения адресов

In [811]:
Address = fact(
    'Address',
    ['Street', 'Type_of_Street',
    'Building_Number', 'Type_of_Building', 'Building',
    'Apartment_Number'] 
            )

#Функция для нормализации "дома" в поле Type_of_Bulding:
def normalize_type_of_build(text):
    text = re.sub(r'д.?$', 'дом', text)
    return text
    
#Функция для нормализации "улицы" в поле Type_of_Street:
def normalize_street(text):
    text = re.sub(r'ул.?$', 'улица', text)
    text = re.sub(r'пер.?$', 'переулок', text)
    text = re.sub(r'ш.?$', 'шоссе', text)
    text = re.sub(r'пр.?$', 'проезд/проспект', text) #неприятная омонимия: проспект или проезд могут сокращаться как "пр."
    text = re.sub(r'наб.?$', 'набережная', text)
    text = re.sub(r'ал.?$', 'аллея', text)
    return text

#Для обработки названий улиц:
STREET_NAME = pipeline(my_streets)

TYPE_OF_STREET = morph_pipeline(['ул',
                'ул.',
                'улица',
                'пер',
                'пер.',
                'переулок',
                'ш',
                'ш.',
                'шоссе',
                'пр',
                'пр.',
                'проезд',
                'тупик',
                'проспект',
                'наб.',
                'наб',
                'набережная',
                'ал.',
                'ал',
                'аллея'               
                ]).interpretation(Address.Type_of_Street.custom(normalize_street))

#Для обработки "дома"
TYPE_OF_BUILDING = morph_pipeline(['д.',
                    'д',
                    'дом']).interpretation(Address.Type_of_Building.custom(normalize_type_of_build))

BUILDING = morph_pipeline(['корп',
                           'корп.',
                           'корпус'])
        
#Правило, извлекающее адреса: #(ул).Арбат(,) (дом) 15(/43); Староконюшенный пер., д.30
ADDRESS = or_(
        rule( 
            TYPE_OF_STREET.optional(),
            STREET_NAME.interpretation(Address.Street),
            TYPE_OF_STREET.optional(),
            COMMA.optional(),
            TYPE_OF_BUILDING.optional(),
            INT.interpretation(Address.Building_Number.custom(int)), 
            eq('/').optional(),
            INT.optional().interpretation(Address.Apartment_Number.custom(int))
            ),

        #Вариант для случая с "корпусом": ул.Волочаевская, д.2, корп.1, Волочаевская ул. дом 2 корп 1
        rule (
            TYPE_OF_STREET.optional(),
            STREET_NAME.interpretation(Address.Street),
            TYPE_OF_STREET.optional(),
            COMMA.optional(),
            TYPE_OF_BUILDING.optional(),
            INT.interpretation(Address.Building_Number.custom(int)), 
            in_('/,').optional(),
            INT.optional().interpretation(Address.Apartment_Number.custom(int)),
            COMMA.optional(),
            BUILDING,
            INT.interpretation(Address.Building.custom(int))  
            )
        ).interpretation(Address)

In [825]:
show_matches(ADDRESS, *flats_list)

BoxMarkup('text1. ann1. Арбатская м. 1-комн. кв-ру, или м.Кропоткинская 7 мин/п, Староконюшенный пер., д.30. 35/21 кв.м, кухня 10, 10/12 эт, блоч. дома, балкон, тел., хор. сост., с/у совм., окна в тихий сквер, освобождение сразу, продаю, 45 тыс.$. Торг Т 557-00-73 ',
          [Span(70, 96, None)])

Address(Street='Староконюшенный',
        Type_of_Street='переулок',
        Building_Number=30,
        Type_of_Building='дом',
        Building=None,
        Apartment_Number=None)

BoxMarkup('text2. ann2. Арбатская м. 1-комн. кв ру, 5 м/пеш., ул.Арбат дом 15/43; 53/20/13 кв.м, еврорем., 4/6-эт. кирп. дома, д/фон, евростиль, тел , кондиц., стоянка, с мебелью, охрана, свободна, хозяин, 130 тыс.$. Торг. Т.762-55-67, Сергей',
          [Span(51, 69, None)])

Address(Street='Арбат',
        Type_of_Street='улица',
        Building_Number=15,
        Type_of_Building='дом',
        Building=None,
        Apartment_Number=43)

BoxMarkup('text3. ann3. Баррикадная м. 1-комн. кв-ру, ул.Б.Грузинская, д.14, 36/20.2 кв.м, кухня 8.3, 2/14-эт. кирп. дома, балкон, тел., консьержка, свободна, Т.962-30-63 ',
          [Span(43, 64, None)])

...

Address(Street='Верхняя',
        Type_of_Street='улица',
        Building_Number=6,
        Type_of_Building='дом',
        Building=None,
        Apartment_Number=None)

BoxMarkup('text19. ann19. Белорусская м. 1-комн. кв-ру, ул. Бутырский вал, д. 34, 10 м/п; 36/20 кв.м, к. 10, 12/12-пан. дома, с/у совм., окна двор, тел., балкон, хор. сост., домофон, свободна, 28 тыс.$. Торг. Т.299-96-51',
          [Span(45, 69, None)])

Address(Street='Бутырский вал',
        Type_of_Street='улица',
        Building_Number=34,
        Type_of_Building='дом',
        Building=None,
        Apartment_Number=None)

BoxMarkup('text20. ann20. Белорусская м. 1-комн. кв-ру, 6 мин.п., Б.Тишинский пер., 22/15 кв.м, 2/6 эт. кирп, дома, паркет, пот.3.15, домофон, тихо, 18 тыс.$, торг, без посредников. Альтернатива. Т.253-30-92, вечером',
          [Span(55, 78, None)])

Address(Street='Б.Тишинский',
        Type_of_Street='переулок',
        Building_Number=22,
        Type_of_Building=None,
        Building=None,
        Apartment_Number=15)

### НЕРЕШЕННЫЕ ПРОБЛЕМЫ в извлечениях адресов:
    
1. Как исключить срабатывание в таком случае: **м.Кропоткинская 7 мин/п**? (У нас есть омонимичная станции метро улица - Кропоткинская.) 
Пока я просто удалилиа эту улицу из списка, потому что она в Мск более не существует (переименована в Пречистенку). Однако понятно, что это неверное решение (костыль) и будут другие случаи. Хорошо бы уметь прописать в правиле, 
что до TYPE_OF_STREET не может быть токена с ABBRV, то есть сделать такую проверку окружающего контекста, однако я не нашла технической возможности это решить.

2. Список названий улиц не может покрыть все случаи, нужно иметь возможность предусмотреть анализ **"неизвестных" улиц**. 
Для этого можно создать правило, опираясь на большую букву у названия (есть предикат is_capitalized()), на длину названия, видимо, и на части речи, наверное, 
но хотелось бы чтобы такое правило срабатывало только при условии, что не сработали первые два, со списком улиц. Как технически такое реализовать? 


#### В финале сделаем один общий факт с извлечением всего, что мне удалось извлечь выше

In [891]:
"""
Новый корневой факт Advert c полями:
тип (в нашем случае будет только "квартира")
название станции метро
количество комнат
расстояние до ближайшей станции метро
адрес

"""
Advert = fact(
        'Advert',
    ['type', 'metro_station', 'number_of_rooms', 
     'distance_from_metro', 'address'
    ]
            )

#Беру уже созданные мною правила для извлечения:
_METRO_STATION = METRO_STATION.interpretation(Advert.metro_station)
_ROOMS_NUMBER = ROOMS_NUMBER.interpretation(Advert.number_of_rooms)
_DISTANCE_FROM_METRO = DISTANCE_FROM_METRO.interpretation(Advert.distance_from_metro)
_ADDRESS = ADDRESS.interpretation(Advert.address)

#Функция для нормализации всех вариантов написания "квартиры":
def normalize_apartment(text):
    text = re.sub(r'кв[-\s]р[уа]|квартир[ау]', 'квартира', text)
    return text

#Извлечем отдельно "квартиру" и поместим ее в поле type:
APARTMENT = or_(rule 
                    (eq('кв'), 
                     DASH.optional(), 
                     in_('рура')), #кв-ру, кв-ра
                rule(normalized('квартира'))
            ).interpretation(Advert.type.custom(normalize_apartment))

#Правило для извлечения всего вместе: Арбатская м. 1-комн. кв ру, 5 м/пеш., ул.Арбат дом 15/43

ADVERT = or_(
        rule(
            _METRO_STATION,
            _ROOMS_NUMBER,
            APARTMENT,
           COMMA.optional(),
            eq('или').optional(), #Арбатская м. 1-комн. кв-ру, или м.Кропоткинская
            _METRO_STATION.optional(),
           _DISTANCE_FROM_METRO.optional(), #Белорусская м. 1-комн. кв-ру, ул.Верхняя, д.6
           in_(',.'),
            _ADDRESS
            ),
        
        #Вариант, когда нет адреса: Бауманская м. 1-комн. кв-ру, метро пешком
        #На самом деле очень странное объявление, однако чтобы правило не срабатывало там, где не нужно, т. е. где все же после идет адрес, нужно проверить, что дальше не идет адрес
        #Поскольку я не знаю, такой тех возможности, как это проверить, я ставлю обязательным дальше INT... -
        rule(
            _METRO_STATION,
            _ROOMS_NUMBER,
            APARTMENT,
           COMMA.optional(),
            eq('или').optional(),
            _METRO_STATION.optional(),
           _DISTANCE_FROM_METRO, 
            COMMA, 
            INT #затычка
        ),
        
        #Вариант, когда расстояние до метро - в самом конце: Бауманская м. 1-комн кв-ру, Госпитальный пер. д.8; 10 мин/п.
        rule(
            _METRO_STATION,
            _ROOMS_NUMBER,
            APARTMENT,
           COMMA.optional(),
            _ADDRESS,
           in_(',;'),
            _DISTANCE_FROM_METRO,
            ) 
        ).interpretation(Advert)

In [887]:
show_matches(ADVERT, *flats_list)

BoxMarkup('text1. ann1. Арбатская м. 1-комн. кв-ру, или м.Кропоткинская 7 мин/п, Староконюшенный пер., д.30. 35/21 кв.м, кухня 10, 10/12 эт, блоч. дома, балкон, тел., хор. сост., с/у совм., окна в тихий сквер, освобождение сразу, продаю, 45 тыс.$. Торг Т 557-00-73 ',
          [Span(13, 96, None)])

Advert(type='квартира',
       metro_station=Metro_Station(name='Кропоткинская'),
       number_of_rooms=Rooms_Number(number=1),
       distance_from_metro=Distance_from_metro(Walking_Distance='да',
                                               Time=7,
                                               Time_Unit='минута'),
       address=Address(Street='Староконюшенный',
                       Type_of_Street='переулок',
                       Building_Number=30,
                       Type_of_Building='дом',
                       Building=None,
                       Apartment_Number=None))

BoxMarkup('text2. ann2. Арбатская м. 1-комн. кв ру, 5 м/пеш., ул.Арбат дом 15/43; 53/20/13 кв.м, еврорем., 4/6-эт. кирп. дома, д/фон, евростиль, тел , кондиц., стоянка, с мебелью, охрана, свободна, хозяин, 130 тыс.$. Торг. Т.762-55-67, Сергей',
          [Span(13, 69, None)])

Advert(type='квартира',
       metro_station=Metro_Station(name='Арбатская'),
       number_of_rooms=Rooms_Number(number=1),
       distance_from_metro=Distance_from_metro(Walking_Distance='да',
                                               Time=5,
                                               Time_Unit='минута'),
       address=Address(Street='Арбат',
                       Type_of_Street='улица',
                       Building_Number=15,
                       Type_of_Building='дом',
                       Building=None,
                       Apartment_Number=43))

BoxMarkup('text3. ann3. Баррикадная м. 1-комн. кв-ру, ул.Б.Грузинская, д.14, 36/20.2 кв.м, кухня 8.3, 2/14-эт. кирп. дома, балкон, тел., консьержка, свободна, Т.962-30-63 ',
          [Span(13, 64, None)])

...

Advert(type='квартира',
       metro_station=Metro_Station(name='Белорусская'),
       number_of_rooms=Rooms_Number(number=1),
       distance_from_metro=None,
       address=Address(Street='Верхняя',
                       Type_of_Street='улица',
                       Building_Number=6,
                       Type_of_Building='дом',
                       Building=None,
                       Apartment_Number=None))

BoxMarkup('text19. ann19. Белорусская м. 1-комн. кв-ру, ул. Бутырский вал, д. 34, 10 м/п; 36/20 кв.м, к. 10, 12/12-пан. дома, с/у совм., окна двор, тел., балкон, хор. сост., домофон, свободна, 28 тыс.$. Торг. Т.299-96-51',
          [Span(15, 77, None)])

Advert(type='квартира',
       metro_station=Metro_Station(name='Белорусская'),
       number_of_rooms=Rooms_Number(number=1),
       distance_from_metro=Distance_from_metro(Walking_Distance='да',
                                               Time=10,
                                               Time_Unit='минута'),
       address=Address(Street='Бутырский вал',
                       Type_of_Street='улица',
                       Building_Number=34,
                       Type_of_Building='дом',
                       Building=None,
                       Apartment_Number=None))

BoxMarkup('text20. ann20. Белорусская м. 1-комн. кв-ру, 6 мин.п., Б.Тишинский пер., 22/15 кв.м, 2/6 эт. кирп, дома, паркет, пот.3.15, домофон, тихо, 18 тыс.$, торг, без посредников. Альтернатива. Т.253-30-92, вечером',
          [Span(15, 78, None)])

Advert(type='квартира',
       metro_station=Metro_Station(name='Белорусская'),
       number_of_rooms=Rooms_Number(number=1),
       distance_from_metro=Distance_from_metro(Walking_Distance='да',
                                               Time=6,
                                               Time_Unit='минута'),
       address=Address(Street='Б.Тишинский',
                       Type_of_Street='переулок',
                       Building_Number=22,
                       Type_of_Building=None,
                       Building=None,
                       Apartment_Number=15))

### НЕРЕШЕННЫЕ ПРОБЛЕМЫ:
1. Как описать корневой факт, не создавая копий для вложенных правил, чтобы избежать в частности копирований атрибутов: metro_station=Metro_Station?
2. Есть ли какая-то техническая возможность проверять, какой токен идет после определенного правила, при этом не включать его в объект? (Эта проблема у меня уже была при извлечении адресов.)
3. В одном из примеров расстояние до метро уехало далеко по тексту - его я не смогла поймать. Нужно уметь как-то указывать какое-то количество токенов после и уметь ставить тех ограничения на контекст...Вот этот пример:  БАУМАНСКАЯ М. 1-комн. кв-ра, Волочаевская ул. дом 2 корп 1, 34/14,4/кух. 8,5, 12/12 пан.+тех.этаж, 15 м.п., телефон